In [56]:
import requests
from bs4 import BeautifulSoup
from anthropic import Anthropic
import gradio as gr
import ollama

In [58]:
API_KEY="hW6XZRb_NicsdHw-Ckwbsk-ant-api03-ioLViphyX26n0A1unMmM48OZegcvufC7UvzmIVyg_kRSaWFezGdqB4kFHVMQ48IkP5NP44LdwAA"
client=Anthropic(api_key=API_KEY)

In [60]:
def olla(prompt):
   
    model = "llama3.2:1b"
    messages = [{"role": "user", "content": prompt}]
    response = ""  
    for chunk in ollama.chat(
        model=model,
        messages=messages,
        stream=True
    ):
        
        text = chunk['message']['content']
     
        response += text or ""
        
        
        yield response


In [62]:
class Website:
    def __init__ (self,url):
        self.url=url
        response=requests.get(url)
        self.body=response.content
        soup=BeautifulSoup(self.body,'html.parser')
        self.title=soup.title.string if soup.string else "No Title Found"
        for irrelevent in soup.body(["script", "style", "img", "input"]):
            irrelevent.decompose()
        self.text=soup.body.get_text(separator="\n",strip=True)
        
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [64]:
class Website:
    def __init__(self, url): 
        self.url = url
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  
            self.body = response.content
            soup = BeautifulSoup(self.body, 'html.parser')
            self.title = soup.title.string if soup.title and soup.title.string else "No Title Found"
            
           
            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = soup.get_text(separator="\n", strip=True)
        except Exception as e:
            self.title = "Error loading page"
            self.text = f"Error: {str(e)}"
        
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [66]:
def stream_brochure(company_name, url, model):
  
    try:
        prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
        prompt += Website(url).get_contents()
        
        if model == "Claude":
            result = mess(prompt)
        elif model == "LLama 3.2:1b":
            result = olla(prompt) 
        else:
            raise ValueError("Unknown model")
        
        yield from result
    except Exception as e:
        yield f"Error: {str(e)}"

In [68]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:", placeholder="Enter company name"),
        gr.Textbox(label="Landing page URL including http:// or https://", placeholder="https://example.com"),
        gr.Dropdown(["LLama 3.2:1b"], label="Select model", value="LLama")
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    title="Company Brochure Generator",
    description="Generate a company brochure by analyzing their website landing page",
    flagging_mode="never"
)

if __name__ == "__main__":
    view.launch()

/opt/anaconda3/lib/python3.11/site-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: LLama or set allow_custom_value=True.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
